In [25]:
import pandas as pd

# 1. Lecture des fichiers CSV
df1 = pd.read_csv('data\ANCIENNETE_data.xlsx - Sheet1.csv', usecols=['idstd','Anciennete'])
df2 = pd.read_csv('data\Colab_data.xlsx - Sheet1.csv', usecols=['idstd','colab'])
df3 = pd.read_csv('data\inno&secteur.xlsx - Sheet1.csv', usecols=['idstd','Innovation'])

df5 = pd.read_csv('data\IT_data.xlsx - Sheet 1.csv', usecols=['idstd','IT'])
df6 =  pd.read_csv('data\R&D_data.xlsx - Sheet1.csv',usecols=['idstd','R&D', 'Country'])

# 2. Fusion itérative sur la colonne commune 'idstd'
df_merged = df1.merge(df2, on='idstd', how='outer')
df_merged = df_merged.merge(df3, on='idstd', how='outer')
df_merged = df_merged.merge(df5, on='idstd', how='outer')
df_merged = df_merged.merge(df6, on='idstd', how='outer')

In [29]:

# 1. Convertir la colonne en type 'category'
df_merged['Country'] = df_merged['Country'].astype('category')

# 2. Réordonner les catégories pour mettre 'IN' en premier
current_categories = df_merged['Country'].cat.categories.tolist()
if 'IN' in current_categories:
    # Retire 'IN' de la liste
    current_categories.remove('IN')
    # Ajoute 'IN' en première position
    new_order = ['IN'] + current_categories
    # Réordonne
    df_merged['Country'] = df_merged['Country'].cat.reorder_categories(new_order, ordered=False)
else:
    print("Attention, la valeur 'IN' n'existe pas dans la colonne 'country'.")

# 3. Créer les variables binaires en supprimant la première catégorie (ici 'IN')
df_merged = pd.get_dummies(
    df_merged, 
    columns=['Country'], 
    drop_first=True  # va « drop » la première catégorie, donc 'IN'
)



In [23]:
na_count = df_merged.isna().sum()

print(na_count)


idstd            0
Anciennete       0
colab         1858
Innovation       1
IT              50
R&D              8
Country_IN       0
Country_NE       0
Country_PA       0
dtype: int64


In [7]:
df_merged.dropna(inplace=True)
df_merged.describe()


,idstd,colab,Innovation,IT,R&D
count,3754.000000,3754.000000,3754.000000,3754.000000,3754.000000
mean,559821.009057,0.213372,0.968034,0.261321,0.804475
std,16721.896583,0.409743,0.175933,0.439413,0.396657
min,527763.000000,0.000000,0.000000,0.000000,0.000000
25%,562442.000000,0.000000,1.000000,0.000000,1.000000
50%,566099.000000,0.000000,1.000000,0.000000,1.000000
75%,569570.750000,0.000000,1.000000,1.000000,1.000000
max,582137.000000,1.000000,1.000000,1.000000,1.000000


In [14]:
df_merged = df_merged[df_merged['Anciennete'] != 'UNKNOWN']


In [8]:
df_merged.drop(columns=['idstd'], inplace=True)

Normaliser data continues

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_merged['Anciennete'] = scaler.fit_transform(df_merged[['Anciennete']])
print(df_merged.dtypes)


Anciennete    float64
colab         float64
Innovation    float64
IT            float64
R&D           float64
dtype: object


In [16]:
y = df_merged['Innovation']
X = df_merged.drop(columns=['Innovation'])